# Review of classical electronic structure methods 

In order to assess the performance of quantum computing algorithms in addressing the electronic structure problem, we will briefly review a few commonly employed classical electronic structure methods of varying level of approximation. 

## Hartree-Fock

The Hartree-Fock (HF) method employs a **mean-field approximation**, where Coulomb correlation of electrons is neglected. The HF wavefunction is hence restricted to the form of a single Slater determinant, for which the optimal form may be acquired by an optimization of the underlying single particle basis (orbitals). Once the optimal HF orbitals have been found, the HF state may be written as 

$$ | \text{HF} \rangle = \prod_{p \in \text{occ}} \hat a^\dagger_p | \text{vac} \rangle $$

where $\hat a^\dagger_p$ creates an electron in the $p^{\rm{th}}$ optimized HF spin orbital, and  $| \text{vac} \rangle$ is the vacuum state (all spin-orbitals unoccupied). Due to the restriction to a single Slater determinant, the HF energy may be obtained very efficiently and can be applied to very large molecules, however it becomes qualitatively incorrect when **electronic correlations** become sufficiently strong. The HF wavefunction is often used as a starting point in more accurate treatments of electronic correlation. 

## Coupled cluster

The Coupled cluster (CC) method introduces electronic correlation to the wavefunction ansatz by operating on the HF reference state with the exponential of excitation operators

$$|\text{CC} \rangle = e^{\hat T} | \text{HF}\rangle, \quad \hat T = \hat T_1 + \hat T_2 + ...$$

where $\hat T_1 = \sum_{ia} t^{a}_i \hat a^\dagger_a \hat a_i$ are referred to as 'singles', $\hat T_2 = \sum_{ijab} t^{ab}_{ij} \hat a^\dagger_a \hat a^\dagger_b \hat a_i \hat a_j$ as 'doubles', etc. When the excitation rank is truncated to only singles (S) and doubles (D), $\hat T = \hat T_1 + \hat T_2$, the corresponding ansatz is referred to as CCSD. Since the number of possible single and double excitations for a system of $N$ electrons in $M$ orbitals is polynomial in $M$, one can efficiently solve a system of equations to obtain the optimized CCSD amplitudes. CCSD often gives accurate energies and can be applied to modestly sized chemical systems. However, due to its implementation, it can **violate the variational principle** and give energies lower than the ground state.

## Full Configuration Interaction
The full configuration interaction (FCI) method yields the **exact ground state energy** within a given basis set. The FCI wavefunction is written as a linear combination of all possible $N$-particle Slater determinants over the orbital basis
$$|\text{FCI} \rangle =  \left( \sum_{ia} C^{a}_i \hat a^\dagger_a \hat a_i + \sum_{ijab} C^{ab}_{ij} \hat a^\dagger_a \hat a^\dagger_b \hat a_i \hat a_j + ...  \right) | \text{HF} \rangle $$
where the sum includes up to $N$-electron excitations, and hence there are an exponential number of coefficients to optimize in $| \text{FCI} \rangle$. Due to its **exponential complexity**, FCI is often reserved for only the smallest chemical systems. However, it serves as a useful result to compare to when assessing tractable electronic structure methods.

# Benchmarking for electronic PESs in minimal basis (STO-3G)

We will apply HF, CCSD, and FCI to obtaining the PESs for a few molecular dissociation processes in minimal (STO-3G) basis.

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from utility import get_molecular_data, obtain_PES

basis = 'sto-3g'

## Task 1 (b)

In [28]:
from tequila import QubitHamiltonian, Variable, quantumchemistry, gates, PauliString, minimize

In [29]:
from timeit import default_timer as timer

In [30]:
# try another basis
basis = 'sto-3g'
# basis = 'cc-pVDZ'
# basis = 'cc-pVDZ'
# basis = 'cc-pVTZ'
# basis = 'cc-pVQZ'
# basis = 'cc-pV5Z'

In [31]:
bond_lengths = np.linspace(0,100,15)

In [32]:
bond_lengths

array([  0.        ,   7.14285714,  14.28571429,  21.42857143,
        28.57142857,  35.71428571,  42.85714286,  50.        ,
        57.14285714,  64.28571429,  71.42857143,  78.57142857,
        85.71428571,  92.85714286, 100.        ])

In [33]:
get_molecular_data

<function utility.get_molecular_data(mol, geometry, xyz_format=False)>

In [34]:
obtain_PES

<function utility.obtain_PES(molecule, bond_lengths, basis, method)>

In [35]:
h2_mol_data = get_molecular_data('h2', 100, xyz_format=True)
h2_mol_data = quantumchemistry.Molecule(h2_mol_data, basis)

In [36]:
h4_mol_data = get_molecular_data('h4', 85, xyz_format=True)
h4_mol_data = quantumchemistry.Molecule(h4_mol_data, basis)

In [37]:
a = h2_mol_data.compute_energy('hf')
a

-0.5486537676143597

In [38]:
b = h4_mol_data.compute_energy('hf')
b

-1.0951285844015137

In [39]:
h = (-13.6 * 0.0367493 * 2)
h

-0.99958096

In [40]:
sizeconsistency_hf =  a * 2 - b

In [41]:
sizeconsistency_hf

-0.00217895082720565

## H<sub>2</sub>

In [ ]:
bond_lengths = np.linspace(0.2,2.6,15)

In [ ]:
bond_lengths

In [ ]:
#Run FCI
start = timer()
FCI_PES = obtain_PES('h2', bond_lengths, basis, method='fci')
end = timer()

In [ ]:
timefci_h2 = end - start
timefci_h2

In [ ]:
#Run HF
start = timer()
HF_PES = obtain_PES('h2', bond_lengths, basis,  method='hf')
end = timer()

In [ ]:
timehf_h2 = end - start
timehf_h2

In [ ]:
#Run CCSD
start = timer()
CCSD_PES = obtain_PES('h2', bond_lengths, basis,  method='ccsd')
end = timer()

In [ ]:
timeccsd_h2 = end - start
timeccsd_h2

In [ ]:
#Run CISD
start = timer()
CISD_PES = obtain_PES('h2', bond_lengths, basis,  method='cisd')
end = timer()

In [ ]:
timecisd_h2 = end - start
timecisd_h2

In [ ]:
for i in range(len(bond_lengths)):
    print(i)
    print("equal") if CCSD_PES[i] == CISD_PES[i] else print("not equal")
    print("difference: " + str((CCSD_PES[i] - CISD_PES[i])*(10**6)))

In [ ]:
#Plot H2 PESs

plt.title('H2 dissociation, STO-3G')
plt.xlabel('R, Angstrom')
plt.ylabel('E, Hartree')

plt.plot(bond_lengths, FCI_PES, label='FCI')
plt.scatter(bond_lengths, HF_PES, label='HF', color='orange')
plt.scatter(bond_lengths, CCSD_PES, label='CCSD', color='purple')
plt.scatter(bond_lengths, CISD_PES, label='CISD', color='green')

plt.legend()

In [ ]:
#Plot H2 TOE

plt.title('H2 time to execute, STO-3G')
plt.ylabel('T, seconds')

times = [timefci_h2, timehf_h2, timeccsd_h2, timecisd_h2]
plt.hist(timefci_h2, label='FCI')
plt.hist(timehf_h2, label='HF', color='orange')
plt.hist(timeccsd_h2, label='CCSD', color='purple')
plt.hist(timecisd_h2, label='CISD', color='green')
plt.hist(timecisd_h2, density=False, bins=30)  # density=False would make counts
plt.legend()

<img src="figs/testimage.png">

## H<sub>2</sub>O symmetric O-H dissociation

In [ ]:
bond_lengths = np.linspace(0.1,2.3,15)

In [ ]:
#Run FCI
start = timer()
FCI_PES = obtain_PES('h2o', bond_lengths, basis, method='fci')
end = timer()

In [ ]:
timefci_h2o = end - start
timefci_h2o

In [ ]:
#Run HF
start = timer()
HF_PES = obtain_PES('h2o', bond_lengths, basis,  method='hf')
end = timer()

In [ ]:
timehf_h2o = end - start
timehf_h2o

In [ ]:
#Run CCSD
start = timer()
CCSD_PES = obtain_PES('h2o', bond_lengths, basis,  method='ccsd')
end = timer()

In [ ]:
timeccsd_h2o = end - start
timeccsd_h2o

In [ ]:
start = timer()
CISD_PES = obtain_PES('h2o', bond_lengths, basis,  method='cisd')
end = timer()

In [ ]:
timecisd_h2o = end - start
timecisd_h2o

In [ ]:
#Plot H2O PESs

plt.title('H2O symmetric dissociation, STO-3G')
plt.xlabel('R, Angstrom')
plt.ylabel('E, Hartree')

plt.plot(bond_lengths, FCI_PES, label='FCI')
plt.scatter(bond_lengths, HF_PES, label='HF', color='orange')
plt.scatter(bond_lengths, CCSD_PES, label='CCSD', color='purple')
plt.scatter(bond_lengths, CISD_PES, label='CISD', color='green')

plt.legend()

## LiH

In [ ]:
bond_lengths = np.linspace(0.3,2.6,15)

In [ ]:
#Run FCI
start = timer()
FCI_PES = obtain_PES('lih', bond_lengths, basis, method='fci')
end = timer()

In [ ]:
timefci_lhi = end - start
timefci_lhi

In [ ]:
start = timer()
HF_PES = obtain_PES('lih', bond_lengths, basis,  method='hf')
end = timer()

In [ ]:
timehf_lhi = end - start
timehf_lhi

In [ ]:
#Run CCSD
star = timer()
CCSD_PES = obtain_PES('lih', bond_lengths, basis,  method='ccsd')
end = timer()

In [ ]:
timeccsd_lhi = end - start
timeccsd_lhi

In [ ]:
#Plot LiH PESs

plt.title('LiH symmetric dissociation, STO-3G')
plt.xlabel('R, Angstrom')
plt.ylabel('E, Hartree')

plt.plot(bond_lengths, FCI_PES, label='FCI')
plt.scatter(bond_lengths, HF_PES, label='HF', color='orange')
plt.scatter(bond_lengths, CCSD_PES, label='CCSD', color='purple')
plt.legend()

## H<sub>4</sub>

In [ ]:
bond_lengths = np.linspace(0.3,2.4,15)

In [ ]:
#Run FCI
start = timer()
FCI_PES = obtain_PES('h4', bond_lengths, basis, method='fci')
end = timer()

In [ ]:
timefci_h4 = end - start
timefci_h4

In [ ]:
#Run HF
start = timer()
HF_PES = obtain_PES('h4', bond_lengths, basis,  method='hf')
end = timer()

In [ ]:
timehf_h4 = end - start
timehf_h4

In [ ]:
#Run CCSD
start = timer()
CCSD_PES = obtain_PES('h4', bond_lengths, basis,  method='ccsd')
end = timer()

In [ ]:
timeccsd_h4 = end - start
timeccsd_h4

In [ ]:
#Plot H4 PESs

plt.title('H4 symmetric dissociation, STO-3G')
plt.xlabel('R, Angstrom')
plt.ylabel('E, Hartree')

plt.plot(bond_lengths, FCI_PES, label='FCI')
plt.scatter(bond_lengths, HF_PES, label='HF', color='orange')
plt.scatter(bond_lengths, CCSD_PES, label='CCSD', color='purple')
plt.legend()